<a href="https://colab.research.google.com/github/yohannes-taye/AAU_CS_IntroductionToOpenGl/blob/main/transcript.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pydub
!apt-get install ffmpeg
!pip install yt-dlp webvtt-py
!pip install googletrans==4.0.0-rc1
!pip install requests
!pip install pypinyin
!pip install jieba
!pip install genanki
!pip install openai

### List all available subtitles from the video. If youre looking to generate chinese audio cards then look for `zh-Hans`

In [2]:
# Download subtitles (vtt format)
#!yt-dlp --write-auto-sub --sub-lang en --sub-format vtt --skip-download https://www.youtube.com/watch?v=g1LsFSO1mbY&t=2s
!yt-dlp --list-subs https://www.youtube.com/watch?v=g1LsFSO1mbY&t=2s
#should appear as zh-Hans

[youtube] Extracting URL: https://www.youtube.com/watch?v=g1LsFSO1mbY
[youtube] g1LsFSO1mbY: Downloading webpage
[youtube] g1LsFSO1mbY: Downloading tv client config
[youtube] g1LsFSO1mbY: Downloading player 9a279502-main
[youtube] g1LsFSO1mbY: Downloading tv player API JSON
[youtube] g1LsFSO1mbY: Downloading ios player API JSON
[youtube] g1LsFSO1mbY: Downloading m3u8 information
[info] Available automatic captions for g1LsFSO1mbY:
Language        Name                                            Formats
zh-Hans                                                         vtt
en                                                              vtt
ab-zh-Hans      Abkhazian from Chinese (Simplified)             vtt, ttml, srv3, srv2, srv1, json3
aa-zh-Hans      Afar from Chinese (Simplified)                  vtt, ttml, srv3, srv2, srv1, json3
af-zh-Hans      Afrikaans from Chinese (Simplified)             vtt, ttml, srv3, srv2, srv1, json3
ak-zh-Hans      Akan from Chinese (Simplified)              

### Download transcript. Add language using the flag `--sub-lang zh-Hans`

In [3]:
!yt-dlp --write-sub --sub-lang zh-Hans --sub-format vtt --skip-download https://www.youtube.com/watch?v=g1LsFSO1mbY&t=2s


[youtube] Extracting URL: https://www.youtube.com/watch?v=g1LsFSO1mbY
[youtube] g1LsFSO1mbY: Downloading webpage
[youtube] g1LsFSO1mbY: Downloading tv client config
[youtube] g1LsFSO1mbY: Downloading player 753b1819-main
[youtube] g1LsFSO1mbY: Downloading tv player API JSON
[youtube] g1LsFSO1mbY: Downloading ios player API JSON
[youtube] g1LsFSO1mbY: Downloading m3u8 information
[info] g1LsFSO1mbY: Downloading subtitles: zh-Hans
[info] g1LsFSO1mbY: Downloading 1 format(s): 616+251
[info] Writing video subtitles to: 史上最值钱的那些公司 了解一下~ [g1LsFSO1mbY].zh-Hans.vtt
[download] Destination: 史上最值钱的那些公司 了解一下~ [g1LsFSO1mbY].zh-Hans.vtt
[download] 100% of   34.04KiB in 00:00:00 at 1.18MiB/s


### Download EditThisCOokie extention for chrome and export cookie for youtube video. use netscape format.

### Then the script below will download the audio file

Download link:
https://chromewebstore.google.com/detail/editthiscookie-v3/ojfebgpkimhlhcblbalbfjblapadhbol

In [4]:
!yt-dlp --cookies /content/cookie.txt -f bestaudio --extract-audio --audio-format mp3 --audio-quality 0 --output "%(title)s.%(ext)s" https://www.youtube.com/watch?v=g1LsFSO1mbY&t=2s


[youtube] Extracting URL: https://www.youtube.com/watch?v=g1LsFSO1mbY
[youtube] g1LsFSO1mbY: Downloading webpage
[youtube] g1LsFSO1mbY: Downloading tv client config
[youtube] g1LsFSO1mbY: Downloading player 9a279502-main
[youtube] g1LsFSO1mbY: Downloading tv player API JSON
[info] g1LsFSO1mbY: Downloading 1 format(s): 251
[download] Destination: 史上最值钱的那些公司 了解一下~.webm
[download] 100% of   17.85MiB in 00:00:01 at 14.87MiB/s
[ExtractAudio] Destination: 史上最值钱的那些公司 了解一下~.mp3
Deleting original file 史上最值钱的那些公司 了解一下~.webm (pass -k to keep)


In [5]:
import webvtt
import json

# Function to decode unicode escape sequences properly
def decode_unicode_escape(text):
    # Ensuring proper decoding if needed
    return text.encode('utf-8').decode('utf-8')

def vtt_to_json(vtt_file, json_out):
    transcript = []

    # Read the .vtt file using webvtt
    for caption in webvtt.read(vtt_file):
        start = convert_timestamp(caption.start)
        end = convert_timestamp(caption.end)

        # Directly assign text, as it's already decoded correctly by webvtt
        decoded_text = caption.text.strip().replace('\n', ' ')

        # Append the caption data to the transcript
        transcript.append({
            "start": start,
            "end": end,
            "text": decoded_text
        })

    # Write the decoded transcript to a JSON file with proper encoding
    with open(json_out, 'w', encoding='utf-8') as f:
        json.dump(transcript, f, indent=2, ensure_ascii=False)

def convert_timestamp(timestamp):
    h, m, s = timestamp.split(":")
    s, ms = s.split(".")
    return int(h)*3600 + int(m)*60 + int(s) + int(ms)/1000.0

# Example usage
vtt_file = "/content/史上最值钱的那些公司 了解一下~ [g1LsFSO1mbY].zh-Hans.vtt"  # Replace with actual .vtt file path
json_out = "transcript.json"
vtt_to_json(vtt_file, json_out)


### Split audio using provided transcript.

In [6]:
from pydub import AudioSegment
import os
import json

# Load the transcript JSON file
with open('transcript.json', 'r', encoding='utf-8') as f:
    transcript = json.load(f)

# Load the audio file (replace with the actual file name)
audio_file = '/content/史上最值钱的那些公司 了解一下~.mp3'  # Replace with the path to your audio file
audio = AudioSegment.from_file(audio_file)

# Create an output directory to save the audio chunks
output_dir = "/content/split_audio"
os.makedirs(output_dir, exist_ok=True)

# Initialize a variable to hold the combined audio chunks
combined_audio = None
combined_text = ""
chunk_counter = 0

# Loop through the transcript and split the audio in pairs
for i in range(0, len(transcript) - 1, 2):  # Step by 2 to process pairs
    # Get the start and end times for the current and next sentence
    start_ms_1 = int(transcript[i]["start"] * 1000)  # Convert to milliseconds
    end_ms_1 = int(transcript[i]["end"] * 1000)  # Convert to milliseconds
    text_1 = transcript[i]["text"]

    start_ms_2 = int(transcript[i+1]["start"] * 1000)  # Convert to milliseconds
    end_ms_2 = int(transcript[i+1]["end"] * 1000)  # Convert to milliseconds
    text_2 = transcript[i+1]["text"]

    # Extract the audio chunks for both entries
    chunk_1 = audio[start_ms_1:end_ms_1]
    chunk_2 = audio[start_ms_2:end_ms_2]

    # Combine the two chunks
    combined_audio = chunk_1 + chunk_2

    # Combine the text from both entries
    combined_text = text_1 + text_2

    # Save the combined audio chunk as a new file
    combined_filename = f"combined_chunk_{chunk_counter:04d}.mp3"  # File name format
    combined_path = os.path.join(output_dir, combined_filename)
    combined_audio.export(combined_path, format="mp3")

    # Save the corresponding combined text as a .txt file
    with open(os.path.join(output_dir, f"combined_chunk_{chunk_counter:04d}.txt"), 'w', encoding='utf-8') as txt_file:
        txt_file.write(combined_text)

    # Print confirmation for the saved file
    print(f"Saved: {combined_filename}")

    # Increment the counter for the next combined chunk
    chunk_counter += 1



Saved: combined_chunk_0000.mp3
Saved: combined_chunk_0001.mp3
Saved: combined_chunk_0002.mp3
Saved: combined_chunk_0003.mp3
Saved: combined_chunk_0004.mp3
Saved: combined_chunk_0005.mp3
Saved: combined_chunk_0006.mp3
Saved: combined_chunk_0007.mp3
Saved: combined_chunk_0008.mp3
Saved: combined_chunk_0009.mp3
Saved: combined_chunk_0010.mp3
Saved: combined_chunk_0011.mp3
Saved: combined_chunk_0012.mp3
Saved: combined_chunk_0013.mp3
Saved: combined_chunk_0014.mp3
Saved: combined_chunk_0015.mp3
Saved: combined_chunk_0016.mp3
Saved: combined_chunk_0017.mp3
Saved: combined_chunk_0018.mp3
Saved: combined_chunk_0019.mp3
Saved: combined_chunk_0020.mp3
Saved: combined_chunk_0021.mp3
Saved: combined_chunk_0022.mp3
Saved: combined_chunk_0023.mp3
Saved: combined_chunk_0024.mp3
Saved: combined_chunk_0025.mp3
Saved: combined_chunk_0026.mp3
Saved: combined_chunk_0027.mp3
Saved: combined_chunk_0028.mp3
Saved: combined_chunk_0029.mp3
Saved: combined_chunk_0030.mp3
Saved: combined_chunk_0031.mp3
Saved: c

In [2]:
!pip install openai
import os
from openai import OpenAI

def get_translation_breakdown(chinese_sentence: str) -> str:
    # Initialize the OpenAI client with your API key
    client = OpenAI(
        api_key="xai-dXIGonbjZNs90ph3MeXiqSC6MkUJ3foBgKM2HxzyQMFcrN6T8rrCXWamlSIJsZ6LuFpD67rh6e76vm9J",
        base_url="https://api.x.ai/v1",
    )

    # Define the prompt with placeholders for the user input
    prompt = f'''
    "Given the following Chinese sentence, provide the English translation, a breakdown of the words (with each word's Chinese form, pinyin, and its translation), and a list of pinyin for each word in the sentence:

    Input: {chinese_sentence}

    Output:

    English translation of the sentence

    A breakdown of each word in the sentence:

    Word in Chinese: (pinyin) [Translation]

    Pinyin for the whole sentence.

    In addition dont add bold or italic formatting. Just do proper content structure formatting like newline.
    "
    '''

    # Request completion from the AI model
    completion = client.chat.completions.create(
        model="grok-3-beta",
        messages=[
            {"role": "system", "content": prompt},
            {"role": "user", "content": chinese_sentence},
        ],
    )

    # Return the result from the AI model
    return completion.choices[0].message.content

# Example usage:
chinese_input = "不知道小伙伴们的选择题做对了几道呢所以你可以很明显地看到每5到10年"
result = get_translation_breakdown(chinese_input)
print(result)


English translation of the sentence:
I don’t know how many multiple-choice questions the little friends got right, so you can clearly see every 5 to 10 years.

A breakdown of each word in the sentence:

Word in Chinese: 不知道 (bù zhīdào) [don’t know]
Word in Chinese: 小伙伴们 (xiǎo huǒbànmen) [little friends/partners]
Word in Chinese: 的 (de) [possessive particle, of]
Word in Chinese: 选择题 (xuǎnzé tí) [multiple-choice questions]
Word in Chinese: 做对 (zuò duì) [got right/correct]
Word in Chinese: 了 (le) [particle indicating completed action]
Word in Chinese: 几道 (jī dào) [how many (questions)]
Word in Chinese: 呢 (ne) [particle for question or emphasis]
Word in Chinese: 所以 (suǒyǐ) [so/therefore]
Word in Chinese: 你 (nǐ) [you]
Word in Chinese: 可以 (kěyǐ) [can]
Word in Chinese: 很 (hěn) [very]
Word in Chinese: 明显地 (míngxiǎn de) [clearly/obviously]
Word in Chinese: 看到 (kàndào) [see]
Word in Chinese: 每 (měi) [every]
Word in Chinese: 5到10年 (5 dào 10 nián) [5 to 10 years]

Pinyin for the whole sentence:
bù

In [ ]:

import os
from googletrans import Translator
import jieba  # Import jieba for Chinese word segmentation
import time
# Directory where your text files are stored
output_dir = "/content/split_audio"  # Update with the correct path

# Initialize Google Translate API (googletrans library)
translator = Translator()
sleep_duration = 0.2
# Function to translate a word using Google Translate API
def translate_word(word):
    time.sleep(sleep_duration)
    try:
        translated = translator.translate(word, src='zh-cn', dest='en')
        if translated.text:
            return translated.text
        else:
            print(f"Warning: No translation for {word}")
            return f"No translation for {word}"
    except Exception as e:
        print(f"Error translating {word}: {e}")
        return f"Translation error for {word}"



# Function to get Pinyin of a Chinese word using pypinyin
def get_pinyin(word):
    from pypinyin import lazy_pinyin  # Import here to avoid unnecessary import if not used
    pinyin = lazy_pinyin(word)  # Convert the word to Pinyin
    return ' '.join(pinyin)  # Join the list into a string

# Function to process all .txt files and generate word meanings
def process_text_files():
    for filename in os.listdir(output_dir):
        if filename.endswith(".txt"):
            file_path = os.path.join(output_dir, filename)
            with open(file_path, 'r', encoding='utf-8') as f:
                sentence = f.read().strip()  # Read sentence from the file
                sentence_translation =  translate_word(sentence)
                # Use jieba to segment the sentence into words
                words = jieba.lcut(sentence)  # lcut() returns a list of words

                # For each word in the sentence, translate or look up its meaning
                word_meanings = {}
                for word in words:
                    meaning = translate_word(word)  # Get the English translation

                    pinyin = get_pinyin(word)  # Get the Pinyin for the word
                    if meaning:
                        word_meanings[word] = {'meaning': meaning, 'pinyin': pinyin}

                # Prepare the formatted output
                formatted_output = f"Sentence: {sentence}\n"
                formatted_output += f"Translation: {sentence_translation}\n\n"
                for word, info in word_meanings.items():
                    formatted_output += f"{word}: [{info['meaning']}] ({info['pinyin']})\n"

                print(formatted_output)

                # Save the formatted output back to the same file
                # with open(file_path, 'w', encoding='utf-8') as txt_file:
                #     txt_file.write(formatted_output)

                # Print confirmation for the saved file
                print(f"Formatted output saved in: {filename}")

# Call the function to process the text files
process_text_files()


In [3]:
import genanki
import os
import time
from googletrans import Translator
import jieba
from pypinyin import lazy_pinyin  # For getting pinyin

# Directory where your text files are stored
output_dir = "/content/split_audio"  # Update with the correct path

# Initialize Google Translate API (googletrans library)
translator = Translator()

# Audio file directory (where MP3 files are stored)
audio_dir = "/content/split_audio"  # Update with your directory for MP3 files

# Anki Deck Settings
deck_name = "Chinese Learning Deck"
deck_id = 1234567890  # Unique identifier for your deck (must be an integer)

# Function to get Pinyin of a Chinese word using pypinyin
def get_pinyin(word):
    pinyin = lazy_pinyin(word)  # Convert the word to Pinyin
    return ' '.join(pinyin)  # Join the list into a string

# Function to translate a word using Google Translate API
def translate_word(word):
    try:
        translated = translator.translate(word, src='zh-cn', dest='en')
        return translated.text
    except Exception as e:
        print(f"Error translating {word}: {e}")
        return None

# Function to create Anki card front and back
def create_anki_cards(sentence, sentence_translation, word_meanings, audio_filename):
    # Create the front of the card
    front = f"Text: {sentence}<br>Audio: [sound:{audio_filename}]"

    # Prepare the breakdown for the back of the card
    breakdown = ""
    for word, info in word_meanings.items():
        breakdown += f"{word}: [{info['meaning']}] ({info['pinyin']})<br>"

    # Create the back of the card
    back = f"Translation: {sentence_translation}<br>Audio: [sound:{audio_filename}]<br>Breakdown: {breakdown}"

    return front, back
# Function to create Anki card front and back
def create_anki_cards(sentence, sentence_translation, word_meanings, audio_filename):
    # Create the front of the card
    front = f"Text: {sentence}<br>Audio: [sound:{audio_filename}]"

    # Prepare the breakdown for the back of the card
    breakdown = ""
    for word, info in word_meanings.items():
        breakdown += f"{word}: [{info['meaning']}] ({info['pinyin']})<br>"

    # Convert newlines in sentence_translation to <br> to preserve HTML formatting
    sentence_translation = sentence_translation.replace("\n", "<br>")

    # Create the back of the card
    back = f"Translation: {sentence_translation}<br>Audio: [sound:{audio_filename}]<br>Breakdown:<br>{breakdown}"

    return front, back

# Create the Anki deck
def create_anki_deck_():
    my_deck = genanki.Deck(deck_id, deck_name)
    my_model = genanki.Model(
        1607392319,
        'Basic Model with Audio',
        fields=[
            {'name': 'Front'},
            {'name': 'Back'},
        ],
        templates=[
            {
                'name': 'Card 1',
                'qfmt': '{{Front}}',
                'afmt': '{{Back}}',
            },
        ],
        css="""
        .card {
            font-family: Arial, sans-serif;
            font-size: 20px;
            text-align: center;
        }
        """
    )

    # Process each text file and add the corresponding cards to Anki deck
    for filename in os.listdir(output_dir):
        if filename.endswith(".txt"):
            file_path = os.path.join(output_dir, filename)
            with open(file_path, 'r', encoding='utf-8') as f:
                sentence = f.read().strip()  # Read sentence from the file
                sentence_translation = translate_word(sentence)

                # Use jieba to segment the sentence into words
                words = jieba.lcut(sentence)

                word_meanings = {}
                for word in words:
                    meaning = translate_word(word)
                    pinyin = get_pinyin(word)
                    if meaning:
                        word_meanings[word] = {'meaning': meaning, 'pinyin': pinyin}

                # Assume the corresponding audio file is named the same as the text file (but with mp3 extension)
                audio_filename = filename.replace(".txt", ".mp3")
                audio_filepath = os.path.join(audio_dir, audio_filename)

                # Check if the audio file exists
                if os.path.exists(audio_filepath):
                  # Create front and back for Anki card
                  front, back = create_anki_cards(sentence, sentence_translation, word_meanings, audio_filename)

                  # Create an Anki note (card) and add it to the deck
                  my_note = genanki.Note(
                      model=my_model,
                      fields=[front, back],
                      tags=["Chinese_Learning"]
                  )
                  my_deck.add_note(my_note)
                else:
                  print("Audio file does not exist")

    # Save the deck to a file
    genanki.Package(my_deck).write_to_file("/content/Chinese_Learning_Deck.apkg")
    print("Deck created successfully!")

# Call the function to create the Anki deck
create_anki_deck()


Deck created successfully!


In [ ]:
import genanki
import os
import time
from googletrans import Translator
import jieba
import time
from pypinyin import lazy_pinyin  # For getting pinyin

# Directory where your text files are stored
output_dir = "/content/split_audio"  # Update with the correct path

# Initialize Google Translate API (googletrans library)
translator = Translator()

# Audio file directory (where MP3 files are stored)
audio_dir = "/content/split_audio"  # Update with your directory for MP3 files

# Anki Deck Settings
deck_name = "Chinese Learning Deck"
deck_id = 1234567890  # Unique identifier for your deck (must be an integer)

# Function to get Pinyin of a Chinese word using pypinyin
def get_pinyin(word):
    pinyin = lazy_pinyin(word)  # Convert the word to Pinyin
    return ' '.join(pinyin)  # Join the list into a string

# Function to translate a word using Google Translate API
def translate_word(word):
    time.sleep(0.2)
    try:
        translated = translator.translate(word, src='zh-cn', dest='en')
        return translated.text
    except Exception as e:
        print(f"Error translating {word}: {e}")
        return None

# Function to create Anki card front and back
def create_anki_cards(sentence, sentence_translation, word_meanings, audio_filename):
    # Create the front of the card
    front = f"Text: {sentence}<br>Audio: [sound:{audio_filename}]"

    # Prepare the breakdown for the back of the card
    breakdown = ""
    for word, info in word_meanings.items():
        breakdown += f"{word}: [{info['meaning']}] ({info['pinyin']})<br>"

    # Create the back of the card
    back = f"Translation: {sentence_translation}<br>Audio: [sound:{audio_filename}]<br>Breakdown: {breakdown}"

    return front, back

# Create the Anki deck
def create_anki_deck():
    my_deck = genanki.Deck(deck_id, deck_name)
    my_model = genanki.Model(
        1607392319,
        'Basic Model with Audio',
        fields=[
            {'name': 'Front'},
            {'name': 'Back'},
        ],
        templates=[
            {
                'name': 'Card 1',
                'qfmt': '{{Front}}',
                'afmt': '{{Back}}',
            },
        ],
        css="""
        .card {
            font-family: Arial, sans-serif;
            font-size: 20px;
            text-align: center;
        }
        """
    )

    # List to store audio files for inclusion in the package
    audio_files = []

    # Process each text file and add the corresponding cards to Anki deck
    for filename in os.listdir(output_dir):
        if filename.endswith(".txt"):
            file_path = os.path.join(output_dir, filename)
            with open(file_path, 'r', encoding='utf-8') as f:
                sentence = f.read().strip()  # Read sentence from the file
                sentence_translation = translate_word(sentence)

                # Use jieba to segment the sentence into words
                words = jieba.lcut(sentence)

                word_meanings = {}
                for word in words:
                    meaning = translate_word(word)
                    pinyin = get_pinyin(word)
                    if meaning:
                        word_meanings[word] = {'meaning': meaning, 'pinyin': pinyin}

                # Assume the corresponding audio file is named the same as the text file (but with mp3 extension)
                audio_filename = filename.replace(".txt", ".mp3")
                audio_filepath = os.path.join(audio_dir, audio_filename)

                # Check if the audio file exists
                if os.path.exists(audio_filepath):
                    # Add audio file to the list of media files
                    audio_files.append(audio_filepath)

                    # Create front and back for Anki card if the audio file exists
                    front, back = create_anki_cards(sentence, sentence_translation, word_meanings, audio_filename)

                    # Create an Anki note (card) and add it to the deck
                    my_note = genanki.Note(
                        model=my_model,
                        fields=[front, back],
                        tags=["Chinese_Learning"]
                    )
                    my_deck.add_note(my_note)
                else:
                    print(f"Warning: Audio file '{audio_filename}' not found for sentence: {sentence}")

    # Add audio files to the package
    my_package = genanki.Package(my_deck)
    my_package.media_files = audio_files

    # Save the deck to a file
    my_package.write_to_file("/content/Chinese_Learning_Deck_with_audio.apkg")
    print("Deck created successfully with audio!")

# Call the function to create the Anki deck
create_anki_deck()


In [3]:
import genanki
import os
from openai import OpenAI

# Directory where your text files are stored
output_dir = "/content/split_audio"  # Update with the correct path

# Audio file directory (where MP3 files are stored)
audio_dir = "/content/split_audio"  # Update with your directory for MP3 files

# Anki Deck Settings
deck_name = "Chinese Learning Deck"
deck_id = 1234567890  # Unique identifier for your deck (must be an integer)

# Initialize OpenAI client
client = OpenAI(
    api_key="xai-dXIGonbjZNs90ph3MeXiqSC6MkUJ3foBgKM2HxzyQMFcrN6T8rrCXWamlSIJsZ6LuFpD67rh6e76vm9J",
    base_url="https://api.x.ai/v1",
)



# Function to create Anki card front and back
def create_anki_cards(sentence, sentence_translation, word_meanings, audio_filename):
    # Create the front of the card
    front = f"Text: {sentence}<br>Audio: [sound:{audio_filename}]"

    # Prepare the breakdown for the back of the card
    breakdown = ""
    for word, info in word_meanings.items():
        breakdown += f"{word}: [{info['meaning']}] ({info['pinyin']})<br>"

    # Create the back of the card
    back = f"Translation: {sentence_translation}<br>Audio: [sound:{audio_filename}]<br>Breakdown: {breakdown}"

    return front, back

# Create the Anki deck
def create_anki_deck():
    my_deck = genanki.Deck(deck_id, deck_name)
    my_model = genanki.Model(
        1607392319,
        'Basic Model with Audio',
        fields=[
            {'name': 'Front'},
            {'name': 'Back'},
        ],
        templates=[
            {
                'name': 'Card 1',
                'qfmt': '{{Front}}',
                'afmt': '{{Back}}',
            },
        ],
        css="""
        .card {
            font-family: Arial, sans-serif;
            font-size: 20px;
            text-align: center;
        }
        """
    )

    # Process each text file and add the corresponding cards to Anki deck
    for filename in os.listdir(output_dir):
        if filename.endswith(".txt"):
            file_path = os.path.join(output_dir, filename)
            with open(file_path, 'r', encoding='utf-8') as f:
                sentence = f.read().strip()  # Read sentence from the file

                # Use OpenAI to get translation, breakdown, and pinyin
                sentence_translation_and_breakdown = get_translation_breakdown(sentence)

                # The output from OpenAI will directly give us the structure we need
                # We can directly use the AI model's output without any additional reformatting

                # The sentence_translation_and_breakdown contains both the translation and the word breakdown
                sentence_translation = sentence_translation_and_breakdown  # No need to process it further

                # Assume the corresponding audio file is named the same as the text file (but with mp3 extension)
                audio_filename = filename.replace(".txt", ".mp3")
                audio_filepath = os.path.join(audio_dir, audio_filename)

                # Check if the audio file exists
                if os.path.exists(audio_filepath):
                    # Create front and back for Anki card
                    front, back = create_anki_cards(sentence, sentence_translation, {}, audio_filename)
                    print(front)
                    print(back)
                    # Create an Anki note (card) and add it to the deck
                    my_note = genanki.Note(
                        model=my_model,
                        fields=[front, back],
                        tags=["Chinese_Learning"]
                    )
                    my_deck.add_note(my_note)
                else:
                    print("Audio file does not exist")

    # Save the deck to a file
    genanki.Package(my_deck).write_to_file("/content/Chinese_Learning_Deck.apkg")
    print("Deck created successfully!")

# Call the function to create the Anki deck
create_anki_deck()


Text: 而是站在历史的高度这么一窥探你就能看到全球经济背后的一个一个潮流<br>Audio: [sound:combined_chunk_0004.mp3]
Translation: English translation of the sentence:
Instead, standing at the height of history, with just a glimpse, you can see the trends behind the global economy one by one.

A breakdown of each word in the sentence:
- 而是 (ér shì) [instead]
- 站 (zhàn) [stand]
- 在 (zài) [at/in/on]
- 历史 (lì shǐ) [history]
- 的 (de) [of/possessive particle]
- 高度 (gāo dù) [height/level]
- 这么 (zhè me) [this way/so]
- 一 (yī) [one/a]
- 窥探 (kuī tàn) [peek/glimpse/explore]
- 你 (nǐ) [you]
- 就 (jiù) [then/just]
- 能 (néng) [can/able to]
- 看到 (kàn dào) [see]
- 全球 (quán qiú) [global]
- 经济 (jīng jì) [economy]
- 背后 (bèi hòu) [behind]
- 的 (de) [of/possessive particle]
- 一个一个 (yī gè yī gè) [one by one]
- 潮流 (cháo liú) [trend/tide]

Pinyin for the whole sentence:
ér shì zhàn zài lì shǐ de gāo dù zhè me yī kuī tàn nǐ jiù néng kàn dào quán qiú jīng jì bèi hòu de yī gè yī gè cháo liú<br>Audio: [sound:combined_chunk_0004.mp3]<br>Breakdown: 
Tex

### Zip and download generated clips

In [10]:
import shutil
from google.colab import files

# Specify the folder path
folder_path = "/content/split_audio"

# Create a zip file of the folder
shutil.make_archive('/content/split_audio', 'zip', folder_path)

# Provide a download link for the zip file
files.download('/content/split_audio.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
my_deck

NameError: name 'my_deck' is not defined

Function Documentation: remove_files_starting_from_6
Purpose:
The remove_files_starting_from_6 function is designed to remove files from a specified directory that follow the pattern combined_chunk_ and have chunk numbers starting from 0006 onwards. This function can be used to delete all files that match this condition, including .mp3 and .txt files.

In [7]:
import os

# Directory where your files are stored
directory = '/content/split_audio'  # Update with the correct path if needed

# List all files in the directory
files_in_directory = os.listdir(directory)

# Loop through the files and remove those starting from "combined_chunk_0006" and onwards
for file in files_in_directory:
    # Check if the filename matches the pattern for chunk numbers starting from 0006
    if file.startswith("combined_chunk_"):
        # Extract the number after 'combined_chunk_'
        file_number = file.split('_')[2].split('.')[0]  # Extract the numeric part
        if file_number.isdigit() and int(file_number) >= 6:
            file_path = os.path.join(directory, file)
            os.remove(file_path)
            print(f"Removed: {file_path}")

print("Files starting from combined_chunk_0006 and onwards have been removed.")


Removed: /content/split_audio/combined_chunk_0177.mp3
Removed: /content/split_audio/combined_chunk_0082.txt
Removed: /content/split_audio/combined_chunk_0133.mp3
Removed: /content/split_audio/combined_chunk_0072.mp3
Removed: /content/split_audio/combined_chunk_0204.txt
Removed: /content/split_audio/combined_chunk_0182.txt
Removed: /content/split_audio/combined_chunk_0235.txt
Removed: /content/split_audio/combined_chunk_0064.mp3
Removed: /content/split_audio/combined_chunk_0194.mp3
Removed: /content/split_audio/combined_chunk_0025.txt
Removed: /content/split_audio/combined_chunk_0220.mp3
Removed: /content/split_audio/combined_chunk_0192.mp3
Removed: /content/split_audio/combined_chunk_0236.mp3
Removed: /content/split_audio/combined_chunk_0124.txt
Removed: /content/split_audio/combined_chunk_0149.txt
Removed: /content/split_audio/combined_chunk_0016.txt
Removed: /content/split_audio/combined_chunk_0222.txt
Removed: /content/split_audio/combined_chunk_0148.txt
Removed: /content/split_audi